In [1]:
# Cell 1: Create workspace and write all files for EXIM mock demo
import os, json, textwrap, pathlib
WORKDIR = "/content/exim_demo"
pathlib.Path(WORKDIR).mkdir(parents=True, exist_ok=True)

# --- hs_code_mapping.json ---
hs_map = {
  "paracetamol": {"api": "29242990", "formulations": ["30049099"]},
  "acetaminophen": {"api": "29242990", "formulations": ["30049099"]},
  "ibuprofen": {"api": "29163990", "formulations": ["30049099"]},
  "metformin": {"api": "29333997", "formulations": ["30049099"]},
  "amoxicillin": {"api": "29411010", "formulations": ["30041000"]},
  "atorvastatin": {"api": "29025000", "formulations": ["30049099"]},
  "ciprofloxacin": {"api": "29349910", "formulations": ["30041000"]},
  "omeprazole": {"api": "29349990", "formulations": ["30041000"]},
  "azithromycin": {"api": "29419090", "formulations": ["30041000"]},
  "doxycycline": {"api": "29349990", "formulations": ["30041000"]}
}
with open(os.path.join(WORKDIR,"hs_code_mapping.json"),"w") as f:
    json.dump(hs_map,f,indent=2)

# --- mock_trade_data.json ---
mock_trade_data = {
  "29242990": {
    "hs_desc":"Paracetamol API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":5200, "export_volume_mt":1300, "import_value_million_usd":94.3,
        "top_import_sources":[{"country":"China","percent":72},{"country":"USA","percent":15},{"country":"Italy","percent":8},{"country":"Others","percent":5}],
        "yearly_trend":{"2019":4100,"2020":4300,"2021":4700,"2022":5200,"2023":5400}
      }
    }
  },
  "29163990": {
    "hs_desc":"Ibuprofen API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":900,"export_volume_mt":3500,"import_value_million_usd":28.4,
        "top_import_sources":[{"country":"Germany","percent":40},{"country":"China","percent":30},{"country":"India (domestic)","percent":20},{"country":"Others","percent":10}],
        "yearly_trend":{"2019":800,"2020":850,"2021":870,"2022":900,"2023":920}
      }
    }
  },
  "29333997": {
    "hs_desc":"Metformin API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":50,"export_volume_mt":12000,"import_value_million_usd":1.2,
        "top_import_sources":[{"country":"China","percent":25},{"country":"India (domestic)","percent":60},{"country":"Others","percent":15}],
        "yearly_trend":{"2019":10000,"2020":10500,"2021":11000,"2022":11500,"2023":12000}
      }
    }
  },
  "29411010": {
    "hs_desc":"Amoxicillin API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":300,"export_volume_mt":2000,"import_value_million_usd":18.0,
        "top_import_sources":[{"country":"China","percent":45},{"country":"Germany","percent":20},{"country":"India (domestic)","percent":25},{"country":"Others","percent":10}],
        "yearly_trend":{"2019":1800,"2020":1850,"2021":1900,"2022":1950,"2023":2000}
      }
    }
  },
  "29025000": {
    "hs_desc":"Atorvastatin API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":20,"export_volume_mt":8000,"import_value_million_usd":0.6,
        "top_import_sources":[{"country":"India (domestic)","percent":70},{"country":"Italy","percent":15},{"country":"Others","percent":15}],
        "yearly_trend":{"2019":7000,"2020":7400,"2021":7600,"2022":7900,"2023":8000}
      }
    }
  },
  "29349910": {
    "hs_desc":"Ciprofloxacin API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":120,"export_volume_mt":900,"import_value_million_usd":6.5,
        "top_import_sources":[{"country":"China","percent":50},{"country":"India (domestic)","percent":30},{"country":"Germany","percent":10},{"country":"Others","percent":10}],
        "yearly_trend":{"2019":800,"2020":900,"2021":1000,"2022":1100,"2023":1200}
      }
    }
  },
  "29349990": {
    "hs_desc":"Omeprazole/Doxycycline API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":400,"export_volume_mt":3000,"import_value_million_usd":22.0,
        "top_import_sources":[{"country":"China","percent":35},{"country":"India (domestic)","percent":50},{"country":"Others","percent":15}],
        "yearly_trend":{"2019":2500,"2020":2600,"2021":2700,"2022":2900,"2023":3000}
      }
    }
  },
  "29419090": {
    "hs_desc":"Azithromycin API (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":60,"export_volume_mt":2200,"import_value_million_usd":3.4,
        "top_import_sources":[{"country":"India (domestic)","percent":60},{"country":"China","percent":25},{"country":"Others","percent":15}],
        "yearly_trend":{"2019":2000,"2020":2050,"2021":2100,"2022":2150,"2023":2200}
      }
    }
  },
  "30041000": {
    "hs_desc":"Finished formulations (tablets) generic (mock)",
    "country_data":{
      "India":{
        "import_volume_mt":150,"export_volume_mt":45000,"import_value_million_usd":12.0,
        "top_export_destinations":[{"country":"USA","percent":30},{"country":"Brazil","percent":20},{"country":"Africa","percent":25},{"country":"Others","percent":25}],
        "yearly_trend":{"2019":40000,"2020":41000,"2021":42000,"2022":44000,"2023":45000}
      }
    }
  }
}
with open(os.path.join(WORKDIR,"mock_trade_data.json"),"w") as f:
    json.dump(mock_trade_data,f,indent=2)

# --- risk_calc.py (CORRECTED) ---
risk_calc = textwrap.dedent("""\
from typing import Dict, List

def supplier_concentration_risk(top_sources: List[Dict]) -> (str,int):
    if not top_sources:
        return 'Unknown', None
    top = max(s.get('percent', 0) for s in top_sources)
    if top >= 70:
        return 'High', top
    if top >= 40:
        return 'Medium', top
    return 'Low', top

def domestic_manufacturing_index(import_vol: float, export_vol: float) -> float:
    denom = (import_vol or 0) + (export_vol or 0)
    if denom == 0:
        return None
    return round(export_vol / denom, 3)

def import_dependency(import_vol: float, export_vol: float) -> float:
    denom = (import_vol or 0) + (export_vol or 0)
    if denom == 0:
        return None
    return round(import_vol / denom, 3)

def price_trend_risk(yearly_trend: Dict[str, float]) -> str:
    if not yearly_trend:
        return 'Unknown'
    years = sorted(yearly_trend.keys())
    if len(years) < 2:
        return 'Unknown'

    first = yearly_trend[years[0]]
    last = yearly_trend[years[-1]]

    if first == 0:
        return 'Unknown'

    change_pct = (last - first) / first

    # Corrected logic: Volatility = Risk
    if change_pct < -0.15: # Price dropped by >15%
        return 'High'
    if change_pct > 0.15: # Price increased by >15%
        return 'Medium'
    if -0.15 <= change_pct <= 0.15: # Stable price change
        return 'Low'

    return 'Unknown'


def aggregate_overall_risk(scr: str, dmi: float, ptr: str) -> str:
    score_map = {'Low':0, 'Medium':1, 'High':2, 'Unknown':1}

    # DMI Risk Score logic correction
    dmi_risk_score = 0
    if dmi is None:
        dmi_risk_score = 1 # Treat Unknown DMI as Medium risk
    elif dmi < 0.5:
        dmi_risk_score = 2 # High dependency (DMI < 0.5) is High Risk
    # else: dmi >= 0.5 is Low Risk (0 points)

    score = score_map.get(scr, 1) + dmi_risk_score + score_map.get(ptr, 1)

    if score >= 4:
        return 'High'
    if score >= 2:
        return 'Medium'
    return 'Low'
""")
with open(os.path.join(WORKDIR,"risk_calc.py"),"w") as f:
    f.write(risk_calc)

# --- server.py (FastAPI mock) ---
server_py = textwrap.dedent("""\
from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import JSONResponse
import json, os
from datetime import datetime, timezone, timedelta
from risk_calc import supplier_concentration_risk, domestic_manufacturing_index, import_dependency, price_trend_risk, aggregate_overall_risk

BASE = os.path.dirname(__file__)
with open(os.path.join(BASE,'hs_code_mapping.json')) as f:
    HS = json.load(f)
with open(os.path.join(BASE,'mock_trade_data.json')) as f:
    DB = json.load(f)

app = FastAPI(title='Mock EXIM API', version='0.1')

def find_hs_for_molecule(molecule: str):
    key = molecule.strip().lower()
    if key in HS:
        return HS[key]
    for k,v in HS.items():
        if key in k:
            return v
    return None

@app.get('/exim/lookup')
def lookup(molecule: str = Query(...)):
    entry = find_hs_for_molecule(molecule)
    if not entry:
        raise HTTPException(status_code=404, detail='HS code not found for molecule')
    return {'molecule': molecule, 'hs_code': entry['api'], 'formulations': entry.get('formulations',[])}

@app.get('/exim/api')
def exim_api(hs_code: str = Query(...), country: str = Query('India')):
    item = DB.get(hs_code)
    if not item:
        raise HTTPException(status_code=404, detail='HS code not found in DB')
    country_info = item.get('country_data',{}).get(country)
    if not country_info:
        raise HTTPException(status_code=404, detail=f'No country data for {country}')
    import_v = country_info.get('import_volume_mt',0)
    export_v = country_info.get('export_volume_mt',0)
    top_sources = country_info.get('top_import_sources',[])
    yearly = country_info.get('yearly_trend',{})
    scr, top_pct = supplier_concentration_risk(top_sources)
    dmi = domestic_manufacturing_index(import_v, export_v)
    idp = import_dependency(import_v, export_v)
    ptr = price_trend_risk(yearly)
    overall = aggregate_overall_risk(scr, dmi, ptr)
    fetched_at = (datetime.now(timezone.utc) + timedelta(hours=5,minutes=30)).isoformat()
    resp = {
        'hs_code': hs_code,
        'hs_desc': item.get('hs_desc'),
        'country': country,
        'import_volume_mt': import_v,
        'export_volume_mt': export_v,
        'import_value_million_usd': country_info.get('import_value_million_usd'),
        'top_import_sources': top_sources,
        'yearly_trend': yearly,
        'computed': {
            'supplier_concentration': scr,
            'supplier_top_percent': top_pct,
            'domestic_manufacturing_index': dmi,
            'import_dependency': idp,
            'price_trend_risk': ptr,
            'overall_risk': overall
        },
        'provenance': [{'source':'mock_exim_server','url':f'file://{os.path.join(BASE,"mock_trade_data.json")}#/{hs_code}','fetched_at':fetched_at}]
    }
    return JSONResponse(content=resp)

@app.get('/exim/formulation')
def exim_formulation(hs_code: str = Query(...), country: str = Query('India')):
    return exim_api(hs_code=hs_code, country=country)

if __name__=='__main__':
    import uvicorn
    uvicorn.run('server:app', host='0.0.0.0', port=8005, reload=True)
""")
with open(os.path.join(WORKDIR,"server.py"),"w") as f:
    f.write(server_py)

# --- worker_wrapper.py ---
worker_py = textwrap.dedent("""\
import requests, json
from datetime import datetime, timezone, timedelta

MOCK_BASE = 'http://127.0.0.1:8005'

def llm_like_summary(computed, api_trade):
    top = api_trade.get('top_import_sources', [])
    top_country = top[0]['country'] if top else 'Unknown'
    import_vol = api_trade.get('import_volume_mt')
    export_vol = api_trade.get('export_volume_mt')
    dmi = computed.get('domestic_manufacturing_index')
    overall = computed.get('overall_risk')
    summary = f"{import_vol} MT imported (exports {export_vol} MT); top supplier {top_country}. Domestic index={dmi}. Overall EXIM risk={overall}."
    insights = [f"Top supplier {top_country}", f"Domestic index {dmi}", f"Overall risk {overall}"]
    confidence = 'High' if overall in ('Low','Medium','High') else 'Low'
    return {'summary_paragraph': summary, 'top_insights': insights, 'confidence': confidence}

def call_exim_worker(molecule, country='India'):
    # lookup hs
    r = requests.get(f"{MOCK_BASE}/exim/lookup", params={'molecule': molecule}, timeout=10)
    r.raise_for_status()
    hs = r.json().get('hs_code')
    r2 = requests.get(f"{MOCK_BASE}/exim/api", params={'hs_code': hs, 'country': country}, timeout=10)
    r2.raise_for_status()
    data = r2.json()
    computed = data.get('computed', {})
    api_trade = {
        'hs_code': data.get('hs_code'),
        'hs_desc': data.get('hs_desc'),
        'import_volume_mt': data.get('import_volume_mt'),
        'export_volume_mt': data.get('export_volume_mt'),
        'import_value_million_usd': data.get('import_value_million_usd'),
        'top_import_sources': data.get('top_import_sources'),
        'yearly_trend': data.get('yearly_trend')
    }
    llm_summary = llm_like_summary(computed, api_trade)
    fetched_at = (datetime.now(timezone.utc) + timedelta(hours=5,minutes=30)).isoformat()
    result = {
        'agent':'EXIM',
        'molecule': molecule,
        'country': country,
        'api_trade': api_trade,
        'formulation_trade': {'hs_codes': []},
        'computed_risks': computed,
        'llm_summary': llm_summary,
        'provenance': data.get('provenance', []),
        'metadata': {'generated_at': fetched_at, 'tool_version':'mock-exim-v1'}
    }
    return result

if __name__=='__main__':
    import sys, json
    m = sys.argv[1] if len(sys.argv)>1 else 'Paracetamol'
    print(json.dumps(call_exim_worker(m), indent=2))
""")
with open(os.path.join(WORKDIR,"worker_wrapper.py"),"w") as f:
    f.write(worker_py)

# --- run_demo.sh ---
run_sh = textwrap.dedent("""\
#!/usr/bin/env bash
python3 -m pip install --upgrade pip
pip install fastapi uvicorn requests
echo "Starting FastAPI mock EXIM server on port 8005..."
# Run the server (note: in Colab run in background as shown in next cell)
uvicorn server:app --host 0.0.0.0 --port 8005
""")
with open(os.path.join(WORKDIR,"run_demo.sh"),"w") as f:
    f.write(run_sh)

# --- Dockerfile ---
docker = textwrap.dedent("""\
FROM python:3.11-slim
WORKDIR /app
COPY . /app
RUN pip install --no-cache-dir fastapi uvicorn requests
EXPOSE 8005
CMD ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8005"]
""")
with open(os.path.join(WORKDIR,"Dockerfile"),"w") as f:
    f.write(docker)

# --- langgraph_tool_snippet.yaml ---
lg = textwrap.dedent("""\
# LangGraph HTTP tool node example for EXIM worker
- id: exim_tool_http
  type: http_tool
  name: EXIM Mock HTTP Tool
  input:
    - molecule: string
    - country: string
  request:
    method: GET
    url: "http://localhost:8005/exim/api"
    params:
      hs_code: "{{hs_code}}"
      country: "{{country}}"
  prehook: |
    # Recommended: call /exim/lookup earlier in the graph to resolve molecule -> hs_code
  output_mapping:
    - path: "$.import_volume_mt"
      to: "api_trade.import_volume_mt"
    - path: "$.export_volume_mt"
      to: "api_trade.export_volume_mt"
    - path: "$.computed"
      to: "computed_risks"
    - path: "$.provenance"
      to: "provenance"
notes: |
  Worker design: call lookup -> call /exim/api -> produce computed_risks & llm_summary -> return JSON to Master Agent.
""")
with open(os.path.join(WORKDIR,"langgraph_tool_snippet.yaml"),"w") as f:
    f.write(lg)

# --- README.md ---
readme = textwrap.dedent(f"""\
EXIM Worker Agent Demo (Colab)

Files written to: {WORKDIR}

How to run in Google Colab:

1) Install dependencies and run the FastAPI server in a background cell:
    !pip install fastapi uvicorn requests
    # Run server in background:
    !nohup uvicorn /content/exim_demo/server:app --host 0.0.0.0 --port 8005 &> /content/exim_demo/server.log &

2) Call the worker wrapper from another cell:
    >>> from worker_wrapper import call_exim_worker
    >>> print(call_exim_worker('Paracetamol'))

3) Or test via HTTP:
    >>> import requests
    >>> requests.get('http://127.0.0.1:8005/exim/lookup', params={{'molecule':'Paracetamol'}}).json()
    >>> requests.get('http://127.0.0.1:8005/exim/api', params={{'hs_code':'29242990','country':'India'}}).json()

Included files:
- hs_code_mapping.json
- mock_trade_data.json
- risk_calc.py (CORRECTED LOGIC)
- server.py (FastAPI mock)
- worker_wrapper.py (example wrapper that calls mock server and returns EXIM output schema)
- run_demo.sh (helper script)
- Dockerfile
- langgraph_tool_snippet.yaml

Next steps:
- Replace llm_like_summary in worker_wrapper.py with a real LLM call (OpenAI/LLM) using the small prompt template we discussed.
- Add unit tests for risk_calc functions.
""")
with open(os.path.join(WORKDIR,"README.md"),"w") as f:
    f.write(readme)

print('Wrote files to', WORKDIR)
print('List of files:')
for fn in os.listdir(WORKDIR):
    print('-', fn)

Wrote files to /content/exim_demo
List of files:
- run_demo.sh
- Dockerfile
- hs_code_mapping.json
- langgraph_tool_snippet.yaml
- mock_trade_data.json
- risk_calc.py
- server.py
- worker_wrapper.py
- README.md


In [2]:
# Cell 2: install and run FastAPI server in background
%cd /content/exim_demo
!pip install --quiet fastapi uvicorn requests
# start server in background; logs to server.log
!nohup uvicorn server:app --host 0.0.0.0 --port 8005 &> server.log &
# show server log tail (wait a second if needed)
import time, os
time.sleep(1)
print('Server started; log tail:')
!tail -n +1 server.log | sed -n '1,120p'


/content/exim_demo
Server started; log tail:
INFO:     Started server process [3110]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8005 (Press CTRL+C to quit)


In [3]:
# Cell 3: test endpoints
import requests, json
base = "http://127.0.0.1:8005"
print("Lookup Paracetamol:", requests.get(base + "/exim/lookup", params={"molecule":"Paracetamol"}).json())
print("API data (India):", json.dumps(requests.get(base + "/exim/api", params={"hs_code":"29242990","country":"India"}).json(), indent=2)[:800], "...\n")


Lookup Paracetamol: {'molecule': 'Paracetamol', 'hs_code': '29242990', 'formulations': ['30049099']}
API data (India): {
  "hs_code": "29242990",
  "hs_desc": "Paracetamol API (mock)",
  "country": "India",
  "import_volume_mt": 5200,
  "export_volume_mt": 1300,
  "import_value_million_usd": 94.3,
  "top_import_sources": [
    {
      "country": "China",
      "percent": 72
    },
    {
      "country": "USA",
      "percent": 15
    },
    {
      "country": "Italy",
      "percent": 8
    },
    {
      "country": "Others",
      "percent": 5
    }
  ],
  "yearly_trend": {
    "2019": 4100,
    "2020": 4300,
    "2021": 4700,
    "2022": 5200,
    "2023": 5400
  },
  "computed": {
    "supplier_concentration": "High",
    "supplier_top_percent": 72,
    "domestic_manufacturing_index": 0.2,
    "import_dependency": 0.8,
    "price_trend_risk": "Medium",
    "overall_risk": "High"
  },
  "provenance": [
   ...



In [4]:
# Cell 4: call the worker wrapper (Python wrapper that returns worker JSON)
%cd /content/exim_demo
import json
from worker_wrapper import call_exim_worker
res = call_exim_worker("Paracetamol", country="India")
print(json.dumps(res, indent=2))
# Save output for inspection
with open("sample_exim_paracetamol.json","w") as f:
    json.dump(res, f, indent=2)
print("Saved sample_exim_paracetamol.json")


/content/exim_demo
{
  "agent": "EXIM",
  "molecule": "Paracetamol",
  "country": "India",
  "api_trade": {
    "hs_code": "29242990",
    "hs_desc": "Paracetamol API (mock)",
    "import_volume_mt": 5200,
    "export_volume_mt": 1300,
    "import_value_million_usd": 94.3,
    "top_import_sources": [
      {
        "country": "China",
        "percent": 72
      },
      {
        "country": "USA",
        "percent": 15
      },
      {
        "country": "Italy",
        "percent": 8
      },
      {
        "country": "Others",
        "percent": 5
      }
    ],
    "yearly_trend": {
      "2019": 4100,
      "2020": 4300,
      "2021": 4700,
      "2022": 5200,
      "2023": 5400
    }
  },
  "formulation_trade": {
    "hs_codes": []
  },
  "computed_risks": {
    "supplier_concentration": "High",
    "supplier_top_percent": 72,
    "domestic_manufacturing_index": 0.2,
    "import_dependency": 0.8,
    "price_trend_risk": "Medium",
    "overall_risk": "High"
  },
  "llm_summary": 

In [5]:
# Cell 5: display key metrics in a table for multiple molecules
%cd /content/exim_demo
from worker_wrapper import call_exim_worker
molecules = ["Paracetamol","Ibuprofen","Metformin","Amoxicillin","Atorvastatin","Ciprofloxacin"]
rows = []
for m in molecules:
    try:
        r = call_exim_worker(m)
        cr = r['computed_risks']
        rows.append({
            'molecule': m,
            'hs_code': r['api_trade']['hs_code'],
            'import_mt': r['api_trade']['import_volume_mt'],
            'export_mt': r['api_trade']['export_volume_mt'],
            'top_supplier': r['api_trade']['top_import_sources'][0]['country'] if r['api_trade']['top_import_sources'] else None,
            'supplier_pct': cr.get('supplier_top_percent'),
            'domestic_index': cr.get('domestic_manufacturing_index'),
            'overall_risk': cr.get('overall_risk')
        })
    except Exception as e:
        rows.append({'molecule':m, 'error':str(e)})
import pandas as pd
pd.DataFrame(rows)


/content/exim_demo


,molecule,hs_code,import_mt,export_mt,top_supplier,supplier_pct,domestic_index,overall_risk
0,Paracetamol,29242990,5200,1300,China,72,0.200,High
1,Ibuprofen,29163990,900,3500,Germany,40,0.795,Low
2,Metformin,29333997,50,12000,China,60,0.996,Medium
3,Amoxicillin,29411010,300,2000,China,45,0.870,Low
4,Atorvastatin,29025000,20,8000,India (domestic),70,0.998,Medium
5,Ciprofloxacin,29349910,120,900,China,50,0.882,Medium


from matplotlib import pyplot as plt
_df_0['import_mt'].plot(kind='hist', bins=20, title='import_mt')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['export_mt'].plot(kind='hist', bins=20, title='export_mt')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['supplier_pct'].plot(kind='hist', bins=20, title='supplier_pct')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['hs_code'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='import_mt', y='hs_code', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['top_supplier'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='import_mt', y='top_supplier', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['overall_risk'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='import_mt', y='overall_risk', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)